In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt
import torch

sys.path.append('/home/ovoinov/work/sk3d/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.camera_models.central_generic import CentralGeneric
from skrgbd.calibration.camera_models import load_from_colmap_txt
from skrgbd.data.processing.undistortion.redistorter import Redistorter, redist_grid_from_cam_models
from skrgbd.data.dataset.scene_paths import ScenePaths

In [ ]:
scene_name = 'dragon'
scene_paths = ScenePaths(scene_name, data_dir='/mnt/datasets/sk3d/dataset')

device = 'cuda:1'
dtype = torch.double

In [ ]:
# cam_name, mode = 'tis_right', 'rgb'
# sensor = 'tis_left'
# sensor = 'real_sense_rgb'
# sensor = 'real_sense_ir'
# sensor = 'real_sense_ir_right'
# sensor = 'kinect_v2_rgb'
# sensor = 'kinect_v2_ir'
# sensor = 'phone_left_rgb'
# sensor = 'phone_left_ir'
# sensor = 'phone_right_rgb'
# sensor = 'phone_right_ir'
cam_name, mode = 'stl_right', 'rgb'

# FIXME load projection model properly --- add from_state_dict for CentralGeneric
# src_model = scene_paths.cam_model(cam_name, mode, var)
# src_model = CentralGeneric(
#     '/mnt/datasets/sk3d/dataset/calibration/tis_right/rgb/intrinsics.yaml',
#     '/mnt/datasets/sk3d/calibration/raw_calibration/small_scale_sphere/tis_right_inverse_grid.pt',
# )
src_model = CentralGeneric(
    '/mnt/datasets/sk3d/calibration/raw_calibration/small_scale_sphere/stl_right_intrinsics.yaml',
    '/mnt/datasets/sk3d/calibration/raw_calibration/small_scale_sphere/stl_right_inverse_grid.pt',
)
    
dst_model = scene_paths.cam_model(cam_name, mode, 'pinhole')
dst_model = load_from_colmap_txt(dst_model)

src_model = src_model.to(device, dtype)
dst_model = dst_model.to(device, dtype)
uv, dev = redist_grid_from_cam_models(src_model, dst_model)

In [ ]:
def show_error(err, figsize=(30, 10), scale='log', vmax_p=100):
    r"""Visualizes undistortion error. See docs for `redist_grid_from_cam_models` for details."""
    plt.figure(figsize=figsize)
    if scale == 'log':
        vis = err.log10()
        vis = vis.where(err > 0, vis[vis.isfinite()].min())
    else:
        vis = err
    vmax = np.percentile(vis, vmax_p)
    plt.imshow(vis, vmax=vmax)
    plt.colorbar(shrink=.5)
    plt.title('Log of undistortion error in meters')

show_error(dev)

In [ ]:
src_model = src_model.to('cpu')
redistorter = Redistorter.from_cam_models(src_model, uv.permute(1, 2, 0))

In [ ]:
# undist_pt = scene_paths.undist_model(cam_name, mode)
undist_pt = '/tmp/undist.pt'
redistorter.save(undist_pt)